In [1]:
from __future__ import print_function

import pickle
from xml.dom import minidom
from bs4 import BeautifulSoup
import numpy as np

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding

from keras.callbacks import *

C:\Users\Ranet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
doc = minidom.parse('DATA/fixed_xml.xml')

headlines_dom = doc.getElementsByTagName('ArticleHeadline')
articles_dom = doc.getElementsByTagName('ArticleText')
#cleantext = BeautifulSoup(raw_html, "lxml").text
headlines = [headline.firstChild.data for headline in headlines_dom]
articles = [BeautifulSoup(article.firstChild.data, "lxml").text for article in articles_dom]

print(len(headlines))
print(len(articles))

print('Headline:', headlines[-1], '\n')
print('Article:\n', articles[-1])

97415
97415
Headline: Kristiine linnaosa kortermajas süttis jõulupuu põlema, tuld võttis kogu toa sisustus 

Article:
 Täna pärastlõunal läks väga napilt, et ühe tavalise ning toreda pere jaoks oleks kogu aastavahetus võtnud hoopis kurva pöörde. Täna kell 16.19 sai häirekeskus teate, et Kristiines Sõpruse puiesteel on kortermaja ühes teise korruse korteris süttinud elutoas põlema nulg. Tulekahju sai alguse nulu külge riputatud ning põlema süüdatud säraküünaldest. Nulg oli kuiv ning kinnitatud selliselt, et puul vett polnud. Mõne hetkega süttis nulg ise põlema nagu säraküünal. Tuld võttis juba kogu toa sisustus. Tuleõnnetuse ajal oli kodus viis inimest, kolm täiskasvanut ja kaks last. Nähes, et tulekahju kustutamine pole enam võimalik, varjus kogu pere magamistuppa. Perel ei õnnestunud põgenemiseks magamistoa akent avada, pereisa lõi lapse kirjutuslauaga akna katki. Päästjate saabudes oli kogu pere roninud aknast kõrvaloleva autosalongi katusele. Päästjad aitasid nad sealt alla. Kaks me

In [3]:
training_text = []
training_data = []
for i, (input_text, target_text) in enumerate(zip(articles, headlines)):
    if input_text not in training_text:
        training_text.append(input_text)
        training_data.append([input_text.replace('\t', '').replace('\n', ''),
                              target_text.replace('\t', '').replace('\n', '')])

print(len(training_data))
print(training_data[0])

5200
['USAs New Yorgis suri täna Venemaa Föderatsiooni alaline esindaja\xa0ÜRO juures Vitali Tšurkin, teatas Vene välisministeerium. Vitali Tšurkin suri tööpostil, kirjutas Interfax. Homme tähistanuks ta\xa065aastast sünnipäeva. Tšurkin sündis Moskvas 1952. aastal. 1974. aastal lõpetas ta Moskva rahvusvaheliste suhete riikliku instituudi. Ta oli ajalooteaduste kandidaat. 1990. aastate algul töötas ta Vene välisministeeriumi informatsioonivalitsuse ülemana ja välisministri asetäitjana. 1994. aastast oli ta Venemaa suursaadik Belgia kuningriigis ning 1998. aastast Kanadas. Venemaa alaline esindaja ÜRO juures oli ta alates 2006. aastast. Tšurkini karjääris oli oluliseks hetkeks, kui ta noore Nõukogude diplomaadina andis Tšernobõli tuumakatastroofi kohta tunnistusi USA kongressi ees, kirjutas ERRi uudisteportaal. Tegemist oli ajaloos esimese korraga, kui Nõukogude Liidu ametiisik USA esindajatekoja komitee ees tunnistusi andis. Siis veel noor ja küllaltki vähetuntud Tšurkin valiti sellesse

In [5]:
with open("DATA/training_data.txt", "wb") as fp:
    pickle.dump(training_data, fp)

In [2]:
with open("DATA/training_data.txt", "rb") as fp:
    training_data = pickle.load(fp)

In [7]:
with open('DATA/articles_headlines.txt', 'w', encoding='utf8') as f:
    for i, (input_text, target_text) in enumerate(training_data):
        f.write(input_text.strip().replace('\t', '').replace('\n', '') + '\t'
                + target_text.strip().replace('\t', '').replace('\n', '')  + '\n')

In [8]:
print(len(training_data[0]))

2


In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'DATA/articles_headlines2.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
#with open(data_path, 'r', encoding='utf-8') as f:
#    lines = f.read().split('END')
    #lines2 = f.read().split('\t')


#for i, line in enumerate(lines[: min(num_samples, len(lines) - 1)]):
#        input_text, target_text = line.split('\t')
for i, (input_text, target_text) in enumerate(training_data):
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = ' \t ' + target_text + ' \n '
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts), len(target_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 5200 5200
Number of unique input tokens: 158
Number of unique output tokens: 116
Max sequence length for inputs: 29627
Max sequence length for outputs: 182


In [4]:
vocab_size = 50000
text_len = 200
headline_len = 15

#def tokenization_processing(x_to_tok, x_val_to_tok):
tokenizer = Tokenizer(num_words=vocab_size,
                                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
                                   lower=True,
                                   split=" ",
                                   char_level=False,
                                   oov_token='oov')
tok_headline = target_texts
tok_text = input_texts

tokenizer.fit_on_texts(tok_headline + tok_text)
true_vocab_size = len(tokenizer.word_index)
print('true vocab size:',true_vocab_size)

tok_headline = pad_sequences(tokenizer.texts_to_sequences(tok_headline), maxlen=headline_len)
tok_text = pad_sequences(tokenizer.texts_to_sequences(tok_text), maxlen=text_len)

print(target_texts[0])
print(input_texts[0])
print('Headline:',tok_headline[0])
print('Text:',tok_text[0])

true vocab size: 179058
 	 Suri Vene suursaadik ÜRO juures Vitali Tšurkin 
 
USAs New Yorgis suri täna Venemaa Föderatsiooni alaline esindaja ÜRO juures Vitali Tšurkin, teatas Vene välisministeerium. Vitali Tšurkin suri tööpostil, kirjutas Interfax. Homme tähistanuks ta 65aastast sünnipäeva. Tšurkin sündis Moskvas 1952. aastal. 1974. aastal lõpetas ta Moskva rahvusvaheliste suhete riikliku instituudi. Ta oli ajalooteaduste kandidaat. 1990. aastate algul töötas ta Vene välisministeeriumi informatsioonivalitsuse ülemana ja välisministri asetäitjana. 1994. aastast oli ta Venemaa suursaadik Belgia kuningriigis ning 1998. aastast Kanadas. Venemaa alaline esindaja ÜRO juures oli ta alates 2006. aastast. Tšurkini karjääris oli oluliseks hetkeks, kui ta noore Nõukogude diplomaadina andis Tšernobõli tuumakatastroofi kohta tunnistusi USA kongressi ees, kirjutas ERRi uudisteportaal. Tegemist oli ajaloos esimese korraga, kui Nõukogude Liidu ametiisik USA esindajatekoja komitee ees tunnistusi andis

In [5]:
print(tok_headline.shape)    
decoder_input_data = np.zeros((len(input_texts), headline_len), dtype='float32')
for i, ex in enumerate(tok_headline):
    for j, ex_val in enumerate(ex):
        if j > 0:
            decoder_input_data[i, j - 1] = tok_headline[i, j]
print(tok_headline[0])
print(decoder_input_data[0])

(5200, 15)
[    0     0     0     0     0     0    19   727   348  7153  2027   232
  5472 20454    20]
[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.9000e+01
 7.2700e+02 3.4800e+02 7.1530e+03 2.0270e+03 2.3200e+02 5.4720e+03
 2.0454e+04 2.0000e+01 0.0000e+00]


In [15]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

MemoryError: 

In [17]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

print(encoder_input_data[0][1])
print(decoder_input_data[0][1])
print(decoder_target_data[0][1])

print(''.join([list(target_token_index.keys())[list(target_token_index.values()).index(
    np.argmax(i))] for i in decoder_input_data[0]]))
print(''.join([list(target_token_index.keys())[list(target_token_index.values()).index(
    np.argmax(i))] for i in decoder_target_data[0]]))

NameError: name 'encoder_input_data' is not defined

In [6]:
with open("DATA/training_data_proc.txt", "wb") as fp:
    pickle.dump([target_texts, input_texts, decoder_input_data, tok_headline, tok_text, tokenizer], fp)

In [9]:
vocab_size = 50000
text_len = 200
headline_len = 15

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

with open("DATA/training_data_proc.txt", "rb") as fp:
    decoder_input_data, tok_headline, tok_text = pickle.load(fp)

In [10]:
num_encoder_tokens = text_len
num_decoder_tokens = headline_len

tok_text  = tok_text.reshape(5200, 200)
tok_headline = tok_headline.reshape(5200, 15)
decoder_input_data = decoder_input_data.reshape(5200, 15)
#tok_text = np.expand_dims(tok_text, -1)
#tok_headline = np.expand_dims(tok_headline, -1)
#decoder_input_data = np.expand_dims(decoder_input_data, -1)
print(tok_text.shape)
print(tok_headline.shape)
print(decoder_input_data.shape)

(5200, 200)
(5200, 15)
(5200, 15)


In [24]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(vocab_size, latent_dim)(encoder_inputs)
x, state_h, state_c = LSTM(latent_dim,
                           return_state=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(vocab_size, latent_dim)(decoder_inputs)
x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(vocab_size, activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [27]:
def generator(features, labels, batch_size):
    # Create empty arrays to contain batch of features and labels
    batch_features_enc = np.zeros((batch_size, num_encoder_tokens))
    batch_features_dec = np.zeros((batch_size, num_decoder_tokens))
    batch_labels = np.zeros((batch_size, num_decoder_tokens, vocab_size))
    #import pdb;pdb.set_trace()
    while True:
        for i in range(batch_size):
#             import pdb;pdb.set_trace()
#             batch_features_enc[i] = features[0][i]
#             batch_features_dec[i] = features[1][i]
#             print(batch_features_enc.shape)
#             print(batch_features_dec.shape)
#             print(batch_labels.shape)
            batch_labels[i] = to_categorical(labels[i], num_classes=vocab_size)
#         print(batch_labels.shape)
        yield [batch_features_enc, batch_features_dec], batch_labels

In [29]:
# Run training
model.compile(optimizer='adam', loss='categorical_crossentropy')
# model.fit([tok_text, decoder_input_data], tok_headline,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)

model.fit_generator(generator([tok_text, decoder_input_data], tok_headline, batch_size),
                    steps_per_epoch=tok_text.shape[0] / batch_size, nb_epoch=100)

# Save model
#model.save('s2s.h5')

C:\Users\Ranet\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if __name__ == '__main__':
C:\Users\Ranet\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=81.25, epochs=100)`
  if __name__ == '__main__':


Epoch 1/100
82/81 [==============================] - 53s 649ms/step - loss: 3.8095
Epoch 2/100
82/81 [==============================] - 50s 604ms/step - loss: 3.5411
Epoch 3/100
82/81 [==============================] - 50s 604ms/step - loss: 3.3431
Epoch 4/100
82/81 [==============================] - 50s 611ms/step - loss: 3.2233
Epoch 5/100
82/81 [==============================] - 50s 609ms/step - loss: 3.1463
Epoch 6/100
82/81 [==============================] - 50s 609ms/step - loss: 3.0868
Epoch 7/100
82/81 [==============================] - 50s 614ms/step - loss: 3.0414
Epoch 8/100
82/81 [==============================] - 49s 602ms/step - loss: 3.0044
Epoch 9/100
82/81 [==============================] - 49s 596ms/step - loss: 2.9722
Epoch 10/100
82/81 [==============================] - 50s 605ms/step - loss: 2.9457
Epoch 11/100
82/81 [==============================] - 49s 600ms/step - loss: 2.9199
Epoch 12/100
82/81 [==============================] - 49s 602ms/step - loss: 2.8969
E

In [31]:
#model.save('DATA/100_2.7007.h5')
model.save_weights('DATA/100_2.7007_weights.h5')

In [51]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

ValueError: Layer lstm_19 expects 5 inputs, but it received 3 input tensors. Input received: [<tf.Tensor 'input_25:0' shape=(?, ?) dtype=float32>, <tf.Tensor 'input_26:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'input_27:0' shape=(?, 256) dtype=float32>]

In [26]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [29]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [30]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Wow!
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Wow!
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Wow!
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Stop!
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Stop!
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Stop!
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Go on.
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Go on.
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Go on.
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Hello!
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: I see.
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: I see.
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: I try.
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: I won!
Decoded sentence: Ma ei saa sinu enda.

-
Input sentence: Oh no!
Decoded sent